# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096808


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:26,  1.05it/s]

Rendering models:  10%|█         | 3/30 [00:05<00:49,  1.84s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:49,  1.91s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:35,  1.40s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:14<00:01,  3.30it/s]

Rendering models:  90%|█████████ | 27/30 [00:14<00:01,  2.99it/s]

Rendering models:  93%|█████████▎| 28/30 [00:15<00:00,  2.35it/s]

Rendering models: 100%|██████████| 30/30 [00:15<00:00,  2.77it/s]

not-logged-in-3b0b929f4475fd276561    0.000200
Jorgearcelarreta                      0.000208
ethanth                               0.000289
framosuw                              0.049245
Kartoon1x                             0.000199
greenowlgirl1                         0.000893
jnarayanbvg                           0.012300
awright5                              0.003135
xStormmyy                             0.003080
AaronWallage117                       0.005591
not-logged-in-a3d7379de3dae98540ca    0.001143
mmcdonough4                           0.001206
KjellN                                0.000444
not-logged-in-b48f5b396bb5986801e0    0.002398
AdamMorse                             0.000151
Ellaaa666                             0.000775
not-logged-in-04ed9ae880f18762cd9e    0.007443
helios1235                            0.000259
w7250369                              0.001524
wgoltz                                0.000150
r.sundstrom3                          0.000216
IdeasInSpace 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())